 Этап подготовки данных играет важную роль в Data Science. Однако этот этап не пройдёт успешно без тщательного анализа зависимостей в данных. Для успешного преобразования таблицы  необходимо иметь представление о том, как признаки связаны между собой и с целевым признаком, который вы хотели бы предсказать. Поэтому в первой части модуля мы рассмотрим ***основные инструменты, позволяющие производить такой анализ, а также методы сортировки данных***.

   Другая проблема заключается в том, что ваши данные не всегда могут быть представлены в виде одной таблицы. Такое нередко случается при работе с выгрузками из целой базы данных. Например, к вам в руки могут попасть несколько таблиц с информацией о клиентах, об их заказах и о поставщиках. Однако вашей целью может быть анализ данных о поставках в целом при использовании всей предоставленной информации. ***В таком случае таблицы необходимо объединить в единую структуру*** — об этом мы и поговорим во второй части модуля.

                              Цели данного модуля:

- Научиться сортировать данные.
- Получить навыки группировки данных и анализа групп.
-  своить методы построения сводных таблиц и научиться с их помощью исследовать зависимости в признаках.
- Научиться работать с данными, представленными в виде нескольких таблиц, и познакомиться с методами их объединения.
                                    ВСПОМНИМ, С КАКИМИ ДАННЫМИ МЫ РАБОТАЕМ

В этом модуле мы продолжим нашу работу с датасетом о продажах объектов недвижимости в Мельбурне и его пригородах.


В прошлом модуле мы совершили множество преобразований над нашей таблицей — давайте вспомним их:

- удалили столбцы index и Coordinates;
- создали признак средней площади одной комнаты MeanRoomsSquare и ввели коэффициент соотношения площади здания к площади участка — AreaRatio;
- преобразовали признак даты продажи Date в формат datetime и создали на его основе следующие столбцы: номер месяц продажи (MonthSale), номер дня недели - продажи (WeekdaySale), «признак-мигалку» выходного дня (Weekend);
- заменили признак года постройки здания YearBuilt на его возраст AgeBuilding;
- извлекли из признака адреса объекта Address новый признак подтипа улицы StreetType и удалили столбец с адресом;
- уменьшили число уникальных наименований агентств по недвижимости (SellerG), а также число пригородов (Suburb);
- выделили категориальные признаки и преобразовали их в тип данных сategory;
- заменили сокращённые названия категорий признака типа объекта Type на их полные названия (h — house, t —  townhouse, u — unit).

In [1]:
# Прочитаем датасет и выведем первые пять его строк:

import pandas as pd

melb_df = pd.read_csv('data/melb_data_fe.csv')
melb_df.head()

,Suburb,Rooms,Type,Price,Method,SellerG,Date,Distance,Postcode,Bedroom,...,Longtitude,Regionname,Propertycount,MeanRoomsSquare,AreaRatio,MonthSale,AgeBuilding,WeekdaySale,StreetType,Weekend
0,Abbotsford,2,house,1480000.0,S,Biggin,2016-03-12,2.5,3067,2,...,144.9984,Northern Metropolitan,4019,25.20,-0.231707,3,46,5,St,1
1,Abbotsford,2,house,1035000.0,S,Biggin,2016-04-02,2.5,3067,2,...,144.9934,Northern Metropolitan,4019,15.80,-0.327660,4,116,5,St,1
2,Abbotsford,3,house,1465000.0,SP,Biggin,2017-04-03,2.5,3067,3,...,144.9944,Northern Metropolitan,4019,18.75,0.056338,4,117,0,St,0
3,Abbotsford,3,house,850000.0,PI,Biggin,2017-04-03,2.5,3067,3,...,144.9969,Northern Metropolitan,4019,15.75,0.145455,4,47,0,other,0
4,Abbotsford,4,house,1600000.0,VB,Nelson,2016-04-06,2.5,3067,3,...,144.9941,Northern Metropolitan,4019,17.75,0.083969,4,2,2,St,0


Файл подгрузился верно, однако есть один нюанс. Давайте посмотрим на информацию о столбцах с помощью метода info():

In [2]:
display(melb_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13580 entries, 0 to 13579
Data columns (total 26 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Suburb           13580 non-null  object 
 1   Rooms            13580 non-null  int64  
 2   Type             13580 non-null  object 
 3   Price            13580 non-null  float64
 4   Method           13580 non-null  object 
 5   SellerG          13580 non-null  object 
 6   Date             13580 non-null  object 
 7   Distance         13580 non-null  float64
 8   Postcode         13580 non-null  int64  
 9   Bedroom          13580 non-null  int64  
 10  Bathroom         13580 non-null  int64  
 11  Car              13580 non-null  int64  
 12  Landsize         13580 non-null  float64
 13  BuildingArea     13580 non-null  float64
 14  CouncilArea      12211 non-null  object 
 15  Lattitude        13580 non-null  float64
 16  Longtitude       13580 non-null  float64
 17  Regionname  

None

Обратите внимание, что наше преобразование столбцов к типам datetime и category «слетело».

? Почему это произошло?

Ответ на самом деле очень прост: csv-файл не хранит в себе информацию о типах данных столбцов, поэтому при чтении Pandas автоматически определяет тип данных столбца. Не забывайте об этом, обмениваясь преобразованными данными с вашими коллегами.

А пока предлагаем вам самим совершить преобразование столбцов к нужным типам данных ↓

Преобразуйте столбец Date в формат datetime и выделите квартал (quarter) продажи объектов недвижимости. Найдите второй по популярности квартал продажи. В качестве ответа запишите число объектов, проданных в этом квартале.

 - Подсказка (1 из 3): Чтобы преобразовать дату в формат datetime, используйте функцию pd.to_datetime().
 - Подсказка (2 из 3): С помощью атрибута акссесора dt quarter выделите квартал, а с помощью метода value_counts() посчитайте частоту для каждого из кварталов.
 - Подсказка (3 из 3): Чтобы найти число зданий, проданных во втором по популярности квартале, можно воспользоваться iloc.

In [18]:
melb_df['Date'] = pd.to_datetime(melb_df['Date'])
quarters = melb_df['Date'].dt.quarter
display(quarters.value_counts())# а можно сразу display(quarters.value_counts().iloc[1])

3    4873
2    4359
4    2329
1    2019
Name: Date, dtype: int64

In [20]:
display(quarters.value_counts().iloc[1])# в квадратных скобках 1 означает второй по популярнойсти квартал

4359

Преобразуйте все столбцы, в которых меньше 150 уникальных значений, в тип данных category, исключив из преобразования столбцы Date, Rooms, Bedroom, Bathroom, Car.
В качестве ответа запишите результирующее количество столбцов, которые имеют тип данных category.
 - Подсказка (1 из 1): Число столбцов, имеющих тип category, можно узнать с помощью метода info().

In [38]:
melb_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13580 entries, 0 to 13579
Data columns (total 26 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   Suburb           13580 non-null  category      
 1   Rooms            13580 non-null  category      
 2   Type             13580 non-null  category      
 3   Price            13580 non-null  float64       
 4   Method           13580 non-null  category      
 5   SellerG          13580 non-null  category      
 6   Date             13580 non-null  datetime64[ns]
 7   Distance         13580 non-null  float64       
 8   Postcode         13580 non-null  int64         
 9   Bedroom          13580 non-null  int64         
 10  Bathroom         13580 non-null  int64         
 11  Car              13580 non-null  int64         
 12  Landsize         13580 non-null  float64       
 13  BuildingArea     13580 non-null  float64       
 14  CouncilArea      12211 non-null  categ

In [43]:
# Задаём список столбцов, которые мы не берём в рассмотрение (cols_to_exclude)
col_of_exclusif = ['Date', 'Rooms', 'Bedroom', 'Bathroom', 'Car'] 
#  Задаём порог уникальных значений столбца max_unique_count
max_unique_count = 150
for col in melb_df.columns: # В цикле перебираем имена столбцов
    # если число уникальных категорий меньше заданного порога и имён 
    # столбцов нет в списке cols_to_exclude, 
    if melb_df[col].nunique() < max_unique_count and col not in col_of_exclusif:
        # с помощью метода astype() приводим столбец к типу данных category
        melb_df[col] = melb_df[col].astype('category') # преобразуем тип столбца
melb_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13580 entries, 0 to 13579
Data columns (total 26 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   Suburb           13580 non-null  category      
 1   Rooms            13580 non-null  category      
 2   Type             13580 non-null  category      
 3   Price            13580 non-null  float64       
 4   Method           13580 non-null  category      
 5   SellerG          13580 non-null  category      
 6   Date             13580 non-null  datetime64[ns]
 7   Distance         13580 non-null  float64       
 8   Postcode         13580 non-null  int64         
 9   Bedroom          13580 non-null  int64         
 10  Bathroom         13580 non-null  int64         
 11  Car              13580 non-null  int64         
 12  Landsize         13580 non-null  float64       
 13  BuildingArea     13580 non-null  float64       
 14  CouncilArea      12211 non-null  categ